### Question 2: Are women more likely to complete secondary education in some countries than others? In the coming years, what percentage of women overall and by country, do we expect to enroll in secondary education? What factors indicate whether or not a women completes secondary education?

### Import packages and data

In [38]:
#import packages

# general
import numpy as np
import pandas as pd
import time

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# visualization
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

import xgboost as xgb
from xgboost import XGBRegressor

In [26]:
#load in the dataset
df = pd.read_csv('transformed_data.csv')
df.head()

,Unnamed: 0,Year,A woman can be head of household in the same way as a man (1=yes; 0=no),A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),A woman can travel outside her home in the same way as a man (1=yes; 0=no),...,Country Name_Ukraine,Country Name_United Arab Emirates,Country Name_United Kingdom,Country Name_United States,Country Name_Uruguay,Country Name_Uzbekistan,"Country Name_Venezuela, RB",Country Name_Viet Nam,Country Name_West Bank and Gaza,Country Name_Zimbabwe
0,0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,-0.7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#remove the country columns so that we can focus on more important predictors

columns_to_remove = set()
prev_prefix = 'Country'

for col in df.columns:
    prefix = col.split('_')[0]
    
    if prefix == prev_prefix:
        columns_to_remove.add(col)
    
    prev_prefix = prefix

df = df.drop(columns=columns_to_remove)
df = df.drop(columns=['Country Name_Algeria', 'Unnamed: 0'])
df

,Year,A woman can be head of household in the same way as a man (1=yes; 0=no),A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),A woman can travel outside her home in the same way as a man (1=yes; 0=no),A woman can work at night in the same way as a man (1=yes; 0=no),...,The law is free of legal provisions that require a married woman to obey her husband (1=yes; 0=no),The law prohibits discrimination in access to credit based on gender (1=yes; 0=no),The law prohibits discrimination in employment based on gender (1=yes; 0=no),The law provides for the valuation of nonmonetary contributions (1=yes; 0=no),The mandatory retirement age for men and women is the same (1=yes; 0=no),There are periods of absence due to childcare accounted for in pension benefits (1=yes; 0=no),There is legislation specifically addressing domestic violence (1=yes; 0=no),There is paid parental leave (1=yes; 0=no),"Vocational and Technical enrolment (% of total secondary enrolment), total",Share of STEM Graduates
0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,-0.623169,-0.123466
1,-0.7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,-0.560996,0.884704
2,0.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.553385,1.215693
3,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.447417,1.619732
4,0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-0.445124,1.235555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,0.8,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,-0.798411,0.890969
1067,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,-1.181184
1068,0.1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,-0.614411
1069,0.2,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,-0.595872


### Prepare the data for the models

In [28]:
#create X and y dataframes
X = df.drop(columns='School enrollment, secondary, female (% gross)')
y = df[['School enrollment, secondary, female (% gross)']]

In [29]:
#split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)
y_test

,"School enrollment, secondary, female (% gross)"
396,0.004843
227,0.115674
673,0.000000
702,0.000000
643,-1.464595
...,...
35,0.000000
870,-1.331799
503,0.008743
472,-0.075033


#### Feature engineering

In [7]:
#initialize sfs, use linear regression as the baseline model for this question
sfs = SequentialFeatureSelector(estimator = LinearRegression(),
                                n_features_to_select = "auto",
                                direction = 'forward',
                                scoring = 'neg_mean_squared_error',
                                cv = 10)

#fit the data to sfs
sfs = sfs.fit(X_train, y_train)

#retrieve the and print the names of the selected features
feature_names = np.array(df.columns.difference(['School enrollment, secondary, female (% gross)']))
selected_feature_names = feature_names[sfs.get_support()].tolist()
print("Selected features:", selected_feature_names)

# transform X_train and X_test to include only the selected features
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

# display the shape of transformed X_train_selected and X_test_selected
print("Transformed X_train shape:", X_train_selected.shape)
print("Transformed X_test shape:", X_test_selected.shape)

Selected features: ['A woman can get a job in the same way as a man (1=yes; 0=no)', 'A woman can open a bank account in the same way as a man (1=yes; 0=no)', 'A woman can register a business in the same way as a man (1=yes; 0=no)', 'A woman can travel outside her home in the same way as a man (1=yes; 0=no)', 'A woman can work in a job deemed dangerous in the same way as a man (1=yes; 0=no)', 'A woman has the same rights to remarry as a man (1=yes; 0=no)', 'Adolescent fertility rate (births per 1,000 women ages 15-19)', 'Age dependency ratio (% of working-age population)', 'Criminal penalties or civil remedies exist for sexual harassment in employment (1=yes; 0=no)', 'Dismissal of pregnant workers is prohibited (1=yes; 0=no)', 'GDP per capita (Current US$)', 'Inflation, consumer prices (annual %)', 'Length of paid parental leave for father (calendar days)', 'Life expectancy at age 60, female (years)', 'Paid leave is available to fathers (1=yes; 0=no)', 'Population ages 65 and above, fem

### Function for the Models 

In [30]:
def question2(model_type, X_train, y_train):

    # Step 1: Initialize a model object
    if model_type == 'Linear Regression':
      model = LinearRegression()
    elif model_type == 'Ridge':
      model = Ridge()
    elif model_type == 'Random Forest':
      model = RandomForestRegressor()
    elif model_type== 'XGBoost':
      model = XGBRegressor()
    else:
      print('Please specify the model type')

    # Step 2: Train the model
    model.fit(X_train, y_train)

    # Step 3: Make predictions
    y_pred_train = model.predict(X_train)

    # Step 4: Evaluate the model performance

    ## MSE
    #evaluate the model using MSE and R squared
    mse_train = -np.mean(cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error'))
    r_sq_train = np.mean(cross_val_score(model, X_train, y_train, cv=10, scoring='r2'))

    #print the MSE and R_squared
    print('MSE (Train): ', round(mse_train, 3))
    print('R-Squared (Train): ', round(r_sq_train, 3))

    return mse_train, r_sq_train

In [44]:
def question2nocv(model_type, X_train, y_train, X_test, y_test):

    # Step 1: Initialize a model object
    if model_type == 'Linear Regression':
      model = LinearRegression()
    elif model_type == 'Ridge':
      model = Ridge()
    elif model_type == 'Lasso':
      model = Lasso()
    elif model_type == 'Random Forest':
      model = RandomForestRegressor()
    elif model_type == 'Gradient Boosting':
      model = GradientBoostingRegressor()
    elif model_type== 'XGBoost':
      model = XGBRegressor()
    else:
      print('Please specify the model type')

    # Step 2: Train the model
    model.fit(X_train, y_train)

    # Step 3: Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Step 4: Evaluate the model performance

    ## MSE
    #evaluate the model using MSE and R squared
    mse_train = mean_squared_error(y_train,y_pred_train)
    mse_test = mean_squared_error(y_test,y_pred_test)
    r_sq_train = model.score(X_train, y_train)
    r_sq_test = model.score(X_test, y_test)

    #print the MSE and R_squared
    print('MSE (Train): ', round(mse_train, 3))
    print('MSE (Test): ', round(mse_test, 3))
    print('R-Squared (Train): ', round(r_sq_train, 3))
    print('R-Squared (Test): ', round(r_sq_test, 3))

    return mse_train, mse_test, r_sq_train, r_sq_test

### Model #1: Linear Regression

In [46]:
#running the model
print('Linear Regression:')
mse_train_lr, r_sq_train_lr = question2('Linear Regression', X_train, y_train)
mse_test_lr, r_sq_test_lr = question2('Linear Regression', X_test, y_test)
print('\n')
mse_train_lr, r_sq_train_lr, mse_test_lr, r_sq_test_lr = question2nocv('Linear Regression', X_train, y_train, X_test, y_test)


Linear Regression:
MSE (Train):  0.687
R-Squared (Train):  0.632
MSE (Train):  1.677
R-Squared (Train):  0.007


MSE (Train):  0.564
MSE (Test):  0.788
R-Squared (Train):  0.723
R-Squared (Test):  0.565


#### Hypertuning

We will not be hypertuning the basic linear regression model because there are not parameters to tune.

### Model #2: Ridge Model

For the rest of the models, I used the regular X_train data, not the feature selected. These models do not need to use feature selection.

In [50]:
#running the ridge model
print('Ridge Model:')
mse_train_ridge, r_sq_train_ridge = question2('Ridge', X_train, y_train)

Ridge Model:
MSE (Train):  0.682
R-Squared (Train):  0.635


#### Hypertuning

In [51]:
# Generate a random list of regularization parameters (alphas)
alphas = np.logspace(-2, 8, 100)

# Create a list where you will store the information
model_ridge_coefficients = []
mse_train_ridge_list = []
r2_train_ridge_list = []

# Try each set of alphas in your Ridge model
for a in alphas:
    #Pick the model type by initializing a model object and set the alpha from the list as a model parameter
    model_ridge = Ridge()
    model_ridge.set_params(alpha=a)

    #Train the model by passing some data
    model_ridge.fit(X_train, y_train)

    #Evaluate the model performance
    mse_train_ridge = -np.mean(cross_val_score(model_ridge, X_test, y_test, cv=10, scoring='neg_mean_squared_error'))
    r2_train_ridge = np.mean(cross_val_score(model_ridge, X_test, y_test, cv=10, scoring='r2'))

    # Append all results to the lists so we could look at them and also plot them
    model_ridge_coefficients.append(model_ridge.coef_)
    mse_train_ridge_list.append(mse_train_ridge)
    r2_train_ridge_list.append(r2_train_ridge)

In [52]:
#Find and print the best alpha

# Find the lowest value of MSE train value
min_mse_train_ridge = np.min(mse_train_ridge_list)
print("Min MSE Train: ", min_mse_train_ridge)

# Find the index of the lowest MSE train value
min_mse_train_ridge_index = np.argmin(mse_train_ridge_list)
print("Index of Min MSE train: ", min_mse_train_ridge_index)

# Call alphas with the index of the lowest MSE train value
best_alpha = alphas[min_mse_train_ridge_index]
print("Best alpha: ", best_alpha)

Min MSE Train:  1.1617360593892943
Index of Min MSE train:  32
Best alpha:  17.073526474706906


#### Run the model using the best alpha

In [53]:
# Pick the model type by initializing a model object and set the alpha to the best alpha
model_ridge_best_alpha = Ridge()
model_ridge_best_alpha.set_params(alpha=best_alpha)

# Train the model by passing some data
model_ridge_best_alpha.fit(X_train, y_train)

# Get predictions
y_pred_train_ridge_best_alpha = model_ridge_best_alpha.predict(X_train)
y_pred_test_ridge_best_alpha = model_ridge_best_alpha.predict(X_test)

# Evaluate the model performance
r_squared_train_ridge_best_alpha = np.mean(cross_val_score(model_ridge_best_alpha, X_train, y_train, cv=5, scoring='r2'))
r_squared_test_ridge_best_alpha = model_ridge_best_alpha.score(X_test, y_test)
mse_train_ridge_best_alpha = -np.mean(cross_val_score(model_ridge_best_alpha, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
mse_test_ridge_best_alpha = mean_squared_error(y_test, y_pred_test_ridge_best_alpha)

print("Best Ridge R^2 (Train): ", round(r_squared_train_ridge_best_alpha, 3))
print("Best Ridge MSE (Train): ", round(mse_train_ridge_best_alpha, 3))
print("Best Ridge R^2 (Test): ", round(r_squared_test_ridge_best_alpha, 3))
print("Best Ridge MSE (Test): ", round(mse_test_ridge_best_alpha, 3))

Best Ridge R^2 (Train):  0.663
Best Ridge MSE (Train):  0.668
Best Ridge R^2 (Test):  0.578
Best Ridge MSE (Test):  0.764


### Model #3: Random Forest Regressor

In [36]:
#reshape the target variable
y_train_ensemble = y_train.values.ravel()
y_test_ensemble = y_test.values.ravel()

#running the random forest model
print('Random Forest Regressor:')
mse_train_rf, r_sq_train_rf = question2('Random Forest', X_train, y_train_ensemble)


Random Forest Regressor:
MSE (Train):  0.245
R-Squared (Train):  0.862


#### Hypertuning

In [37]:
# Put the parameters options
parameters = {'max_depth': [5, 10, 20],
              'max_features': ['sqrt', 'log2'],
              'min_samples_split': [2, 5, 10]}

# Initialize the model
rfr = RandomForestRegressor()

#Initialize a GridSearchCV object by passing the model, parameters
model_grid_search = GridSearchCV(rfr, parameters, cv=5)

#Train the GridSearchCV
model_grid_search.fit(X_train, y_train_ensemble)

##Print the best parameters
print("Best Parameters:", model_grid_search.best_params_)

## Save the best model into a separate variable to be used later
rfr_finetuned = model_grid_search.best_estimator_

#Evaluate the model

# Get predictions
y_pred_train_rfr_finetuned = rfr_finetuned.predict(X_train)
y_pred_test_rfr_finetuned = rfr_finetuned.predict(X_test)

## r-squared
r_squared_train_rfr_finetuned = rfr_finetuned.score(X_train, y_train_ensemble)
r_squared_test_rfr_finetuned = rfr_finetuned.score(X_test, y_test_ensemble)

## mse
mse_train_rfr_finetuned = mean_squared_error(y_train_ensemble, y_pred_train_rfr_finetuned)
mse_test_rfr_finetuned = mean_squared_error(y_test_ensemble, y_pred_test_rfr_finetuned)

print('R-squared (Train):', round(r_squared_train_rfr_finetuned, 3))
print('R-squared (Test):', round(r_squared_test_rfr_finetuned, 3))
print("Finetuned Ridge MSE (Train): ", round(mse_train_rfr_finetuned, 3))
print("Finetuned Ridge MSE (Test): ", round(mse_test_rfr_finetuned, 3))

Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2}
R-squared (Train): 0.986
R-squared (Test): 0.818
Finetuned Ridge MSE (Train):  0.029
Finetuned Ridge MSE (Test):  0.33


### Model #4 (New Model): XGBoost

I used XGBoost for the new model since the random forest model was performing the best on these data for this question.

In [16]:
#running the XGBoost model
print('XGBoost Model:')
mse_train_xg, r_sq_train_xg = question2('XGBoost', X_train, y_train)


XGBoost Model:


MSE (Train):  -0.268
R-Squared (Train):  0.845


#### Hypertuning

In [41]:
#Define the parameters
parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1]
}

# Initialize the model
xgboost = XGBRegressor()

#Initialize a GridSearchCV object by passing the model, parameters
model_grid_search_xg = GridSearchCV(xgboost, parameters, cv=10)

#Train the GridSearchCV
model_grid_search_xg.fit(X_train, y_train_ensemble)

##Print the best parameters
print("Best Parameters:", model_grid_search_xg.best_params_)

## Save the best model into a separate variable to be used later
xgboost_finetuned = model_grid_search_xg.best_estimator_

#Evaluate the model

# Get predictions
y_pred_train_xgb_finetuned = xgboost_finetuned.predict(X_train)
y_pred_test_xgb_finetuned = xgboost_finetuned.predict(X_test)

## r-squared
r_squared_train_xgb_finetuned = xgboost_finetuned.score(X_train, y_train_ensemble)
r_squared_test_xgb_finetuned = xgboost_finetuned.score(X_test, y_test_ensemble)

## mse
mse_train_xgb_finetuned = mean_squared_error(y_train_ensemble, y_pred_train_xgb_finetuned)
mse_test_xgb_finetuned = mean_squared_error(y_test_ensemble, y_pred_test_xgb_finetuned)

print('R-squared (Train):', round(r_squared_train_xgb_finetuned, 3))
print('R-squared (Test):', round(r_squared_test_xgb_finetuned, 3))
print("Finetuned XGBoost MSE (Train): ", round(mse_train_xgb_finetuned, 3))
print("Finetuned XGBoost MSE (Test): ", round(mse_test_xgb_finetuned, 3))

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
R-squared (Train): 0.998
R-squared (Test): 0.83
Finetuned XGBoost MSE (Train):  0.003
Finetuned XGBoost MSE (Test):  0.308


### Create a DF from the models

In [54]:
# Create a dictionary to hold evauluation metrics from the models
finetuned_data = {

    'Model': [
        'Linear Regression',
        'Ridge (Hypertuned)',
        'Random Forest (Hypertuned)',
        'XGBoost (Hypertuned)'
        ],


    'R Squared Train': [
        r_sq_train_lr,
        r_squared_train_ridge_best_alpha,
        r_squared_train_rfr_finetuned,
        r_squared_train_xgb_finetuned
        ],

    'R Squared Test': [
        r_sq_test_lr,
        r_squared_test_ridge_best_alpha,
        r_squared_test_rfr_finetuned,
        r_squared_test_xgb_finetuned
        ],

    'MSE Train': [
        mse_train_lr,
        mse_train_ridge_best_alpha,
        mse_train_rfr_finetuned,
        mse_train_xgb_finetuned
        ],

    'MSE Test': [
        mse_test_lr,
        mse_test_ridge_best_alpha,
        mse_test_rfr_finetuned,
        mse_test_xgb_finetuned
    ]
    }

# Create and display a dataframe
finetuned_results_df = pd.DataFrame(finetuned_data)
finetuned_results_df

,Model,R Squared Train,R Squared Test,MSE Train,MSE Test
0,Linear Regression,0.788097,0.564605,0.563901,0.723251
1,Ridge (Hypertuned),0.662649,0.578002,0.668194,0.763849
2,Random Forest (Hypertuned),0.985970,0.817640,0.028587,0.330086
3,XGBoost (Hypertuned),0.998486,0.830043,0.003085,0.307634


- The Random Forest Regressor has a very high r squared for the training data but is a little lower for the r squared for the test data, so there may be concerns of overfitting. 
- The XGBoost Model performed very well in terms of having high r-squareds and low MSEs, but there is some concern for overfitting. Hypertuning the model only very minutely changed the r-squared train but it  improved the r-squared test which is good.
- The ridge model improved a little bit with the hypertuning, but mostly the MSE just decreased after choosing the best alpha.
- The basic linear regression performs pretty well compared to the other models, even compared to the hypertuned models. However, it does have higher MSE.

### References

1. Lab 3 Solutions
2. Lab 6 Solutions
3. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
4. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
5. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
6. Lab 8 Solutions
7. https://www.datacamp.com/tutorial/xgboost-in-python
8. https://machinelearningmastery.com/xgboost-for-regression/